In [1]:
import pandas as pd
df = pd.read_csv('D:/Python_WC/Final_project/Multi-Task_News_Intelligence_System/Data/news.tsv', sep='\t',on_bad_lines='skip')
df.head()


,News ID,Category,Topic,Headline,News body,Title entity,Entity content
0,N10000,sports,soccer,Predicting Atlanta United's lineup against Col...,"Only FIVE internationals allowed, count em, FI...","{""Atlanta United's"": 'Atlanta United FC'}","{'Atlanta United FC': {'type': 'item', 'id': '..."
1,N10001,news,newspolitics,Mitch McConnell: DC statehood push is 'full bo...,WASHINGTON -- Senate Majority Leader Mitch McC...,"{'DC': 'Washington, D.C.'}","{'Washington, D.C.': {'type': 'item', 'id': 'Q..."
2,N10002,news,newsus,Home In North Highlands Damaged By Fire,NORTH HIGHLANDS (CBS13) Fire damaged a home ...,{},{}
3,N10003,news,newspolitics,Meghan McCain blames 'liberal media' and 'thir...,Meghan McCain is speaking out after a journali...,{},{}
4,N10004,news,newsworld,Today in History: Aug 1,"1714: George I becomes King Georg Ludwig, Elec...",{},{}


In [2]:
df_labeled = df[df["Title entity"] != "{}"]

len(df_labeled)

63516

In [3]:
df['News body'][0]

"Only FIVE internationals allowed, count em, FIVE! So first off we should say, per our usual Atlanta United lineup predictions, this will be wrong. Why will it be wrong? Well, aside from the obvious, we still don't have a ton of data points from Frank de Boer in how he prefers to rotate his team for   let's be honest   an inferior competition. We've seen how he rotates (or doesn't rotate) in CONCACAF Champions League play, but that's a bit different because CCL was clearly a priority for the club. We got one glimpse of U.S. Open Cup rotation last week when the team played a home game as the visiting team against the Charleston Battery, but will things change on the actual road against an MLS club? Here's my predicted lineup: Let's talk about it: Kann - Seems like he's the Cup keeper. Simples. CBs - I think Leandro Gonzalez Pirez is likely to be a casualty of the 5-international player limit. Miles Robinson is still young and probably isn't in need of much rest, and as far as internatio

In [4]:
df['Title entity'][0:30]

0             {"Atlanta United's": 'Atlanta United FC'}
1                            {'DC': 'Washington, D.C.'}
2                                                    {}
3                                                    {}
4                                                    {}
5                                                    {}
6         {'Chevrolet': 'Chevrolet', 'V8': 'V8 engine'}
7     {'Trump': 'Donald Trump', "Iran's": 'Iran', 'U...
8     {'Mexican': 'Mexico', 'restaurants': 'Restaura...
9                                                    {}
10    {'American Eagle': 'American Eagle Outfitters'...
11                                                   {}
12           {'NBA': 'National Basketball Association'}
13         {'UK': 'United Kingdom', 'Makeup': 'Makeup'}
14    {'MBTA': 'Massachusetts Bay Transportation Aut...
15                         {'Retirement': 'Retirement'}
16    {"Kevin Durant's": 'Kevin Durant', 'NBA': 'Nat...
17                                              

In [5]:
df_unlabeled = df[df["Title entity"] == "{}"]
len(df_unlabeled)

50246

In [6]:
len(df)

113762

In [7]:
# Data Cleaniing and Preprocessing including Stopword Removal

import re
import string
import pandas as pd
from nltk.corpus import stopwords
import nltk

STOPWORDS = set(stopwords.words('english'))

def clean_text(text):

    # This part handles cases where the input is not a string
    if not isinstance(text,str): 
        return ""
    
    # Remove HTML tags
    text =  re.sub(r"<.*?>"," ",text)

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+"," ",text)

    # Remove emojis
    text = re.sub(
        "[" 
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        "]+", 
        "", 
        text
    )

    #  Remove special symbols except letters, numbers, .,!?, and spaces
    text = re.sub(r"[^a-zA-Z0-9\s.,!?]", " ", text)

    #  Normalize punctuation (keep only .,!?)
    allowed = set(string.ascii_letters + string.digits + " .,!?")
    text = "".join(ch for ch in text if ch in allowed)

    #  Convert to lowercase
    text = text.lower()

    # Normalize whitespace
    text = " ".join(text.split())

     # Remove stopwords
    tokens = [
        word for word in text.split()
        if word not in STOPWORDS
    ]
    
    text = " ".join(tokens)

    return text

In [8]:
# cleaning BEFORE tokenization
df["text"] = df["News body"].apply(clean_text)

In [9]:
texts = df["text"].fillna("").tolist()

In [10]:
texts[20001]

'drifts hail freak storm partially buried tractor trailers cars mexican city guadalajara, bringing families streets play ice piled high despite summer heat. security forces soldiers try clear away ice heavy storm rain hail guadalajara, mexico june 30, 2019. drifts hail freak storm partially buried tractor trailers cars mexican city guadalajara, bringing families streets play ice piled high despite summer heat. reuters fernando carranza residents play top ice. storm sunday blanketed parts mexico second largest city white, reuters images show. soldiers police took streets help clear operations slid slipped knee deep drifts. reuters fernando carranza trucks buried ice heavy storm rain hail guadalajara. jorge david arias, visiting family, said relatives never seen storm like it. arias shot photos showing cars pushed trees force ice. reuters fernando carranza residents play top ice guadalajara. guadalajara, western mexico, usually sees average temperatures june around 88 fahrenheit 31 celsi

In [11]:
import re

PATTERNS = {
    "PERSON": r"\b[A-Z][a-z]+\s[A-Z][a-z]+\b",
    "DATE": r"\b\d{4}-\d{2}-\d{2}|\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{4}\b",
    "ORG": r"\b[A-Z][a-zA-Z ]+(?:Ltd|Limited|Bank|University|Corporation|Agency)\b",
    "MONEY": r"\b₹\s?\d+(?:\.\d+)?|\$\s?\d+(?:\.\d+)?\b"
}

def auto_label(text):
    entities = []

    for label, pattern in PATTERNS.items():
        for m in re.finditer(pattern, text):
            entities.append((m.start(), m.end(), label))

    return (text, {"entities": entities})

In [12]:
TRAINING_DATA = [auto_label(text) for text in texts]

In [13]:
TRAINING_DATA[1]

('washington senate majority leader mitch mcconnell criticized push grant statehood puerto rico washington, d.c. interview fox news laura ingraham. plan make district columbia state give two new democratic senators puerto rico state, would give two new democratic senators. full bore socialism march house. yeah, long majority leader senate, none stuff going anywhere. mcconnell elaborate meant that, specifically, clarify adding additional state would constitute socialism. socialism become hot button topic recent months, election several far left democrats identify socialists, traditional gop leaders lambasting it. worth, 1985, economic theorist karl marx labeled socialism phrase according ability, according needs. wusa9 content partners washington post article explaining depth exactly socialism is, read . mcconnell tweeted grim reaper comes stopping democrats socialist agenda. first time decades, house committee oversight reform hold hearing d.c. statehood july 24. related representative

In [14]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(
    TRAINING_DATA,
    test_size=0.2,
    random_state=42
)

print("Train size:", len(train_data))
print("Test size:", len(test_data))

Train size: 91009
Test size: 22753


In [15]:
pip install spacy

     ---------------------------------------- 0.0/44.6 kB ? eta -:--:--
     ---------------------------------------- 44.6/44.6 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.5.0 requires fsspec[http]<=2025.10.0,>=2023.1.0, but you have fsspec 2026.2.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import spacy
from spacy.training.example import Example
from spacy.util import minibatch, compounding
import random

nlp = spacy.blank("en")

# Add NER component
ner = nlp.add_pipe("ner")

# Add labels
for _, annots in train_data:
    for start, end, label in annots["entities"]:
        ner.add_label(label)

# Disable other pipes
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    nlp.initialize()

    N_EPOCHS = 10
    for epoch in range(N_EPOCHS):
        random.shuffle(train_data)
        losses = {}

        batches = minibatch(train_data, size=compounding(4.0, 64.0, 1.5))

        for batch in batches:
            examples = []
            for text, annots in batch:
                doc = nlp.make_doc(text)
                examples.append(Example.from_dict(doc, annots))

            nlp.update(examples, drop=0.2, losses=losses)

        print(f"Epoch {epoch+1}/{N_EPOCHS}, Losses: {losses}")


Epoch 1/10, Losses: {'ner': 5858.411558624479}
Epoch 2/10, Losses: {'ner': 2.9679269526152876e-34}
Epoch 3/10, Losses: {'ner': 6.785130392611747e-32}
